In [2]:
!pip install transformers -qq

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\Mohamed\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

In [ ]:
# Loading pre-trained model
MODEL_NAME = "asafaya/bert-base-arabic"
model = BertForQuestionAnswering.from_pretrained(MODEL_NAME)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at asafaya/bert-base-arabic and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
data = """\
الغسالة تعتبر جهازًا أساسيًا في أي منزل\
حيث توفر وسيلة سهلة وفعالة لغسل الملابس\
تتوفر الغسالات بمجموعة متنوعة من الأحجام والميزات\
بما في ذلك البرامج المخصصة لأنواع معينة من الأقمشة ودرجات الاتساخ والتجفيف الذاتي\
"""
data

'الغسالة تعتبر جهازًا أساسيًا في أي منزلحيث توفر وسيلة سهلة وفعالة لغسل الملابستتوفر الغسالات بمجموعة متنوعة من الأحجام والميزاتبما في ذلك البرامج المخصصة لأنواع معينة من الأقمشة ودرجات الاتساخ والتجفيف الذاتي'

In [29]:
question = """\
ما هي وظيفة الغسالة ؟\
"""

question = """\
ما هي وظيفة الغسالة في المنزل؟\
"""

question = """\
ما هي الخطوات الأساسية لاستخدام الغسالة بشكل صحيح؟\
"""

question = """\
ما هي الإعدادات التي يجب اختيارها عند استخدام الغسالة وفقًا لنوع الملابس؟\
"""

question

'ما هي الإعدادات التي يجب اختيارها عند استخدام الغسالة وفقًا لنوع الملابس؟'

In [45]:
def get_answer(question, data, model, tokenizer):
    inputs = tokenizer(question, data, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer


In [46]:
def print_answer(question, answer):
    print(f"سؤال: {question:_^85}")
    print(f"جواب: {answer:_^85}")

In [ ]:
result = get_answer(question, data, model, tokenizer)
print_answer(question, result)

سؤال: ما هي وظيفة الغسالة؟
جواب: متنوعة من الاحجام والميزات ، بما في ذلك البرامج المخصصة لانواع معينة من الاقمشة


In [ ]:
# fine tuning

In [29]:
import torch
from transformers import BertTokenizer, BertForMaskedLM, AdamW, get_linear_schedule_with_warmup

In [30]:
file_path = "/content/data.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()

In [31]:
tokenizer = BertTokenizer.from_pretrained("asafaya/bert-base-arabic")
input_text = "ظبت مؤقت المطبخ"

In [32]:
inputs = tokenizer(input_text, padding="max_length", max_length=512, truncation=True, return_tensors="pt")


In [33]:
model = BertForMaskedLM.from_pretrained("asafaya/bert-base-arabic")


Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 3
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=epochs)

In [35]:
for epoch in range(epochs):
    outputs = model(**inputs)
    loss = outputs.loss
    if loss is not None:
        loss.backward()
        optimizer.step()
        scheduler.step()
        model.zero_grad()
    else:
        print("Loss is None. Skipping this batch.")

Loss is None. Skipping this batch.
Loss is None. Skipping this batch.
Loss is None. Skipping this batch.


In [36]:
def generate_answer(text, model, tokenizer, max_length=512):
    inputs = tokenizer(text, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt")
    generated_ids = model.generate(input_ids=inputs.input_ids[:, :max_length], max_length=max_length, num_beams=5, early_stopping=True)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [37]:
input_text = "ظبت مؤقت المطبخ؟"
result = generate_answer(input_text, model, tokenizer)
print(f"Input Text: {input_text}")
print(f"Generated Answer: {result}")

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 512, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


RuntimeError: ignored